# Data Processing in R and Python - Assignment 3

## Imports

In [1]:
import pandas as pd
import numpy as np
import os
import os.path
import sqlite3
import tempfile

## Reading data

In [2]:
Tags = pd.read_csv("/Users/michalderej/Downloads/Tags.csv")
Comments = pd.read_csv("/Users/michalderej/Downloads/Comments.csv")
PostLinks = pd.read_csv("/Users/michalderej/Downloads/PostLinks.csv")
Posts = pd.read_csv("/Users/michalderej/Downloads/Posts.csv")
Badges = pd.read_csv("/Users/michalderej/Downloads/Badges.csv")
Users = pd.read_csv("/Users/michalderej/Downloads/Users.csv")
Votes = pd.read_csv("/Users/michalderej/Downloads/Votes.csv")

## Creating a database and importing data frames
In this part, we create a simple database that will store data frames so that Pandas SQL interpreter can read them.

In [3]:
db = os.path.join(tempfile.mkdtemp(), 'example.db')

if os.path.isfile(db):
    os.remove(db)

# Creating connection to db
conn = sqlite3.connect(db)

# Importing data frames to db
Badges.to_sql("Badges", conn)
PostLinks.to_sql("PostLinks", conn)
Posts.to_sql("Posts", conn)
Tags.to_sql("Tags", conn)
Users.to_sql("Users", conn)
Votes.to_sql("Votes", conn)

## First task
### 1.1 - SQL query

In [4]:
sql_query = pd.read_sql_query("""
SELECT
    Name,
    COUNT(*) AS Number,
    MIN(Class) AS BestClass
FROM Badges
GROUP BY Name
ORDER BY Number DESC LIMIT 10
""", conn)
print(sql_query)

               Name  Number  BestClass
0    Autobiographer   24564          3
1           Student   23930          3
2         Supporter   17826          3
3  Popular Question   14840          3
4       Nice Answer   12406          3
5          Yearling   11281          2
6           Teacher    9746          3
7            Editor    9115          3
8  Notable Question    8613          2
9           Scholar    7224          3


### 1.2 - Pandas query
First, let's see how the data frame looks.

In [5]:
Badges.head()

,Id,UserId,Name,Date,Class,TagBased
0,1,2,Autobiographer,2011-06-21T20:16:48.910,3,False
1,2,3,Autobiographer,2011-06-21T20:16:48.910,3,False
2,3,4,Autobiographer,2011-06-21T20:16:48.910,3,False
3,4,6,Autobiographer,2011-06-21T20:21:49.157,3,False
4,5,8,Autobiographer,2011-06-21T20:21:49.157,3,False


Now, we can create a function that will select only the needed data. First, we choose two columns that we will need: `Name` and `Class`. Then, we create a count of every required value and place it in a `Number` column. Next, we populate the `BestClass` column with minimum `Class` values for each row. The following step is to drop duplicates, once again, select the required columns and group and sort them according to the SQL query. In the end, we reset the index and select only the top 10 values.

In [72]:
df = Badges[["Name", "Class"]]
df["Number"] = df["Name"].map(df["Name"].value_counts())
df["BestClass"] = df.groupby("Name")["Class"].transform("min")
df = df.drop_duplicates()
df = df[["Name", "Number", "BestClass"]].groupby(by=["Name"]).sum()
df = df.sort_values(by=["Number"], ascending=False).head(10)
df = df.reset_index()
print(df)

               Name  Number  BestClass
0    Autobiographer   24564          3
1           Student   23930          3
2         Supporter   17826          3
3  Popular Question   14840          3
4       Nice Answer   12406          3
5          Yearling   11281          2
6           Teacher    9746          3
7            Editor    9115          3
8  Notable Question    8613          2
9           Scholar    7224          3


Let's compare the results:

In [73]:
print(sql_query.equals(df))

True


## Second task
### 2.1 - SQL query

In [74]:
sql_query = pd.read_sql_query("""
SELECT Location, COUNT(*) AS Count
FROM (
    SELECT Posts.OwnerUserId, Users.Id, Users.Location
    FROM Users
    JOIN Posts ON Users.Id = Posts.OwnerUserId
)
WHERE Location NOT IN ('')
GROUP BY Location
ORDER BY Count DESC
LIMIT 10
""", conn)
print(sql_query)

                        Location  Count
0      Christchurch, New Zealand   2765
1                   New York, NY   1788
2         London, United Kingdom   1708
3                             UK   1590
4  Sunshine Coast QLD, Australia   1550
5                      Australia   1183
6              Vancouver, Canada    967
7                    Netherlands    935
8             on the server farm    924
9                   Pennsylvania    921


### 2.2 - Pandas query
First, let's see how the data frames look.

In [78]:
Posts.head()

,Id,PostTypeId,AcceptedAnswerId,CreationDate,Score,ViewCount,Body,OwnerUserId,LastEditorUserId,LastEditDate,...,Tags,AnswerCount,CommentCount,ClosedDate,ContentLicense,FavoriteCount,ParentId,LastEditorDisplayName,CommunityOwnedDate,OwnerDisplayName
0,1,1,393.0,2011-06-21T20:19:34.730,8,578.0,<p>My fiancée and I are looking for a good Car...,9.0,101.0,2011-12-28T21:36:43.910,...,<caribbean><cruising><vacations>,4.0,4,2013-02-25T23:52:47.953,CC BY-SA 3.0,NaN,NaN,NaN,NaN,NaN
1,2,1,NaN,2011-06-21T20:22:33.760,43,3255.0,"<p>This was one of our definition questions, b...",13.0,19400.0,2021-04-27T00:47:20.470,...,<guides><extreme-tourism><amazon-river><amazon...,8.0,4,NaN,CC BY-SA 4.0,5.0,NaN,NaN,NaN,NaN
2,3,2,NaN,2011-06-21T20:24:28.080,15,NaN,<p>One way would be to go through an Adventure...,9.0,NaN,NaN,...,NaN,NaN,2,NaN,CC BY-SA 3.0,NaN,2.0,NaN,NaN,NaN
3,4,1,NaN,2011-06-21T20:24:57.160,8,291.0,<p>Singapore Airlines has an all-business clas...,24.0,693.0,2013-01-09T09:55:22.743,...,<loyalty-programs><routes><ewr><singapore-airl...,1.0,1,NaN,CC BY-SA 3.0,NaN,NaN,NaN,NaN,NaN
4,5,1,770.0,2011-06-21T20:25:56.787,14,476.0,<p>Another definition question that interested...,13.0,101.0,2011-12-28T21:36:18.230,...,<romania><transportation>,5.0,0,NaN,CC BY-SA 3.0,2.0,NaN,NaN,NaN,NaN


In [77]:
Users.head()

,Id,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,AccountId
0,-1,1,2011-06-21T15:16:44.253,Community,2011-06-21T15:16:44.253,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\n\n<p>I'm ...",301,6309,24166,-1.0
1,2,101,2011-06-21T20:10:03.720,Geoff Dalgas,2017-10-09T14:25:03.613,http://stackoverflow.com,"Corvallis, OR",<p>Dev #2 who helped create Stack Overflow cur...,41,1,0,2.0
2,3,101,2011-06-21T20:11:02.490,Nick Craver,2019-11-13T23:54:04.630,https://nickcraver.com/blog/,"Winston-Salem, NC",<p>I am the Architecture Lead for Stack Overfl...,22,1,0,7598.0
3,4,101,2011-06-21T20:12:21.223,Emmett,2019-09-05T02:57:12.997,http://minesweeperonline.com,"San Francisco, CA","<p>co-founder of <a href=""https://airtable.com...",15,1,0,1998.0
4,5,101,2011-06-21T20:15:31.410,Kevin Montrose,2019-05-21T18:43:29.443,https://kevinmontrose.com,"New York, NY, United States","<p><a href=""http://blog.stackoverflow.com/2010...",14,3,0,29738.0


Now, we can create a function that will select only the needed data. First, we select the required columns from `Posts` and `Users` data frames and merge them to a new data frame, dropping NaN's during the process. Next, we add a count column, counting unique `Location` occurrences. Then, once again, we select the required columns, drop duplicates and group by `Location`. The three last steps are to sort the values descendingly, select the top 10 records and reset their indices.

In [131]:
posts_data = Posts[["OwnerUserId"]]
users_data = Users[["Id", "Location"]]
df = pd.merge(posts_data, users_data, left_on="OwnerUserId", right_on="Id").dropna()
df["Count"] = df["Location"].map(df["Location"].value_counts())
df = df[["Location", "Count"]]
df = df.drop_duplicates()
df = df.loc[df["Location"] != "NaN"].groupby(by=["Location"]).sum()
df = df.sort_values(by=["Count"], ascending=False).head(10)
df = df.reset_index()
print(df)

                        Location  Count
0      Christchurch, New Zealand   2765
1                   New York, NY   1788
2         London, United Kingdom   1708
3                             UK   1590
4  Sunshine Coast QLD, Australia   1550
5                      Australia   1183
6              Vancouver, Canada    967
7                    Netherlands    935
8             on the server farm    924
9                   Pennsylvania    921


In [132]:
print(sql_query.equals(df))

True


## Third task
### 3.1 - SQL query

In [133]:
sql_query = pd.read_sql_query("""
SELECT
    Users.AccountId,
    Users.DisplayName,
    Users.Location,
    AVG(PostAuth.AnswersCount) as AverageAnswersCount
FROM (
    SELECT
        AnsCount.AnswersCount,
        Posts.Id,
        Posts.OwnerUserId
    FROM (
        SELECT Posts.ParentId, COUNT(*) AS AnswersCount
        FROM Posts
        WHERE Posts.PostTypeId = 2
        GROUP BY Posts.ParentId
        ) AS AnsCount
    JOIN Posts ON Posts.Id = AnsCount.ParentId
) AS PostAuth
JOIN Users ON Users.AccountId=PostAuth.OwnerUserId
GROUP BY OwnerUserId
ORDER BY AverageAnswersCount DESC
LIMIT 10
""", conn)
print(sql_query)

   AccountId      DisplayName           Location  AverageAnswersCount
0      280.0            csmba  San Francisco, CA                 11.0
1    40811.0           vocaro       San Jose, CA                 11.0
2      204.0             Josh          Australia                 10.0
3    44093.0    Emma Arbogast          Salem, OR                 10.0
4    11758.0         rvarcher  Oklahoma City, OK                  9.0
5    19588.0       JD Isaacks        Atlanta, GA                  8.0
6    20473.0      Jeremy Boyd        Houston, TX                  8.0
7    42364.0         Petrogad               None                  8.0
8    54571.0        Christian               None                  8.0
9    79346.0  Thomas Matthews         California                  8.0


### 3.2 - Pandas query
First, let's see how the data frames look.

In [134]:
Posts.head()

,Id,PostTypeId,AcceptedAnswerId,CreationDate,Score,ViewCount,Body,OwnerUserId,LastEditorUserId,LastEditDate,...,Tags,AnswerCount,CommentCount,ClosedDate,ContentLicense,FavoriteCount,ParentId,LastEditorDisplayName,CommunityOwnedDate,OwnerDisplayName
0,1,1,393.0,2011-06-21T20:19:34.730,8,578.0,<p>My fiancée and I are looking for a good Car...,9.0,101.0,2011-12-28T21:36:43.910,...,<caribbean><cruising><vacations>,4.0,4,2013-02-25T23:52:47.953,CC BY-SA 3.0,NaN,NaN,NaN,NaN,NaN
1,2,1,NaN,2011-06-21T20:22:33.760,43,3255.0,"<p>This was one of our definition questions, b...",13.0,19400.0,2021-04-27T00:47:20.470,...,<guides><extreme-tourism><amazon-river><amazon...,8.0,4,NaN,CC BY-SA 4.0,5.0,NaN,NaN,NaN,NaN
2,3,2,NaN,2011-06-21T20:24:28.080,15,NaN,<p>One way would be to go through an Adventure...,9.0,NaN,NaN,...,NaN,NaN,2,NaN,CC BY-SA 3.0,NaN,2.0,NaN,NaN,NaN
3,4,1,NaN,2011-06-21T20:24:57.160,8,291.0,<p>Singapore Airlines has an all-business clas...,24.0,693.0,2013-01-09T09:55:22.743,...,<loyalty-programs><routes><ewr><singapore-airl...,1.0,1,NaN,CC BY-SA 3.0,NaN,NaN,NaN,NaN,NaN
4,5,1,770.0,2011-06-21T20:25:56.787,14,476.0,<p>Another definition question that interested...,13.0,101.0,2011-12-28T21:36:18.230,...,<romania><transportation>,5.0,0,NaN,CC BY-SA 3.0,2.0,NaN,NaN,NaN,NaN


In [135]:
Users.head()

,Id,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,AccountId
0,-1,1,2011-06-21T15:16:44.253,Community,2011-06-21T15:16:44.253,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\n\n<p>I'm ...",301,6309,24166,-1.0
1,2,101,2011-06-21T20:10:03.720,Geoff Dalgas,2017-10-09T14:25:03.613,http://stackoverflow.com,"Corvallis, OR",<p>Dev #2 who helped create Stack Overflow cur...,41,1,0,2.0
2,3,101,2011-06-21T20:11:02.490,Nick Craver,2019-11-13T23:54:04.630,https://nickcraver.com/blog/,"Winston-Salem, NC",<p>I am the Architecture Lead for Stack Overfl...,22,1,0,7598.0
3,4,101,2011-06-21T20:12:21.223,Emmett,2019-09-05T02:57:12.997,http://minesweeperonline.com,"San Francisco, CA","<p>co-founder of <a href=""https://airtable.com...",15,1,0,1998.0
4,5,101,2011-06-21T20:15:31.410,Kevin Montrose,2019-05-21T18:43:29.443,https://kevinmontrose.com,"New York, NY, United States","<p><a href=""http://blog.stackoverflow.com/2010...",14,3,0,29738.0


Now, we can create a function that will select only the needed data. First, we selected only values for column `ParentId` where `PostTypeId == 2` and dropped NaNs. Then, we added an `AnswerCount` column and selected required columns from `Posts`. The next step was to merge both data tables. After that, we again selected the required columns and dropped duplicates. After these operations, the frame was ready to calculate the mean; that's why we added the column `AverageAnswersCount` that holds this data. Next, we selected only the needed columns and merged the data for the second time. We repeated selecting required columns and dropping duplicates and then sorted the data descendingly, selecting the top 10 values and resetting their indices. After these operations, we obtained equal data frames with row permutations.

In [355]:
ans_count = Posts.loc[Posts["PostTypeId"] == 2][["ParentId"]].dropna()
ans_count["AnswersCount"] = ans_count["ParentId"].map(ans_count["ParentId"].value_counts())
post_auth = Posts[["Id", "OwnerUserId"]]
post_auth = pd.merge(post_auth, ans_count, left_on="Id", right_on="ParentId")
post_auth = post_auth[["AnswersCount", "Id", "OwnerUserId"]].drop_duplicates()
post_auth["AverageAnswersCount"] = post_auth.groupby("OwnerUserId")["AnswersCount"]
df = Users[["AccountId", "DisplayName", "Location"]]
df = pd.merge(df, post_auth, left_on="AccountId", right_on="OwnerUserId")
df = df[["AccountId", "DisplayName", "Location", "AverageAnswersCount"]].drop_duplicates()
df = df.sort_values(by=["AverageAnswersCount"], ascending=False).head(10).reset_index(drop=True)
print(df)

   AccountId      DisplayName           Location  AverageAnswersCount
0    40811.0           vocaro       San Jose, CA                 11.0
1      280.0            csmba  San Francisco, CA                 11.0
2      204.0             Josh          Australia                 10.0
3    44093.0    Emma Arbogast          Salem, OR                 10.0
4    11758.0         rvarcher  Oklahoma City, OK                  9.0
5    42364.0         Petrogad                NaN                  8.0
6    19588.0       JD Isaacks        Atlanta, GA                  8.0
7    54571.0        Christian                NaN                  8.0
8    20473.0      Jeremy Boyd        Houston, TX                  8.0
9    79346.0  Thomas Matthews         California                  8.0


**Attention:** to show that the data is the same, but with row permutations, we will sort both frames and drop their indices.

In [361]:
sql_sorted = sql_query.sort_values(["AverageAnswersCount", "DisplayName"], ascending=[False, False]).reset_index(drop=True)
pd_sorted = df.sort_values(["AverageAnswersCount", "DisplayName"], ascending=[False, False]).reset_index(drop=True)
print(sql_sorted.equals(pd_sorted))

True


## Fourth task
### 4.1 - SQL query

In [386]:
sql_query = pd.read_sql_query("""
SELECT
    Posts.Title,
    UpVotesPerYear.Year,
    MAX(UpVotesPerYear.Count) AS Count
FROM (
        SELECT
            PostId,
            COUNT(*) AS Count,
            STRFTIME('%Y', Votes.CreationDate) AS Year
        FROM Votes
        WHERE VoteTypeId=2
        GROUP BY PostId, Year
    ) AS UpVotesPerYear
JOIN Posts ON Posts.Id=UpVotesPerYear.PostId
WHERE Posts.PostTypeId=1
GROUP BY Year
ORDER BY Year ASC
""", conn)
print(sql_query)

                                                Title  Year  Count
0   OK we're all adults here, so really, how on ea...  2011     70
1     How to successfully haggle / bargain in markets  2012     37
2   Why are airline passengers asked to lift up wi...  2013    103
3   How do you know if Americans genuinely/literal...  2014    179
4   Immigration officer that stopped me at the air...  2015    117
5   I don't know my nationality. How can I visit D...  2016    134
6                        Why prohibit engine braking?  2017    177
7   How can I find restaurants in the USA where ti...  2018    119
8   My name causes an issue with any booking! (nam...  2019    263
9   What's the longest distance that can be travel...  2020    110
10  Positive drug test in Singapore: How long do I...  2021     61


### 4.2 - Pandas query
First, let's see how the data frames look.

In [458]:
Votes.head()

,Id,PostId,VoteTypeId,CreationDate,UserId,BountyAmount
0,1,1,2,2011-06-21T00:00:00.000,NaN,NaN
1,2,1,2,2011-06-21T00:00:00.000,NaN,NaN
2,3,2,2,2011-06-21T00:00:00.000,NaN,NaN
3,4,3,2,2011-06-21T00:00:00.000,NaN,NaN
4,5,5,5,2011-06-21T00:00:00.000,13.0,NaN


In [459]:
Posts.head()

,Id,PostTypeId,AcceptedAnswerId,CreationDate,Score,ViewCount,Body,OwnerUserId,LastEditorUserId,LastEditDate,...,Tags,AnswerCount,CommentCount,ClosedDate,ContentLicense,FavoriteCount,ParentId,LastEditorDisplayName,CommunityOwnedDate,OwnerDisplayName
0,1,1,393.0,2011-06-21T20:19:34.730,8,578.0,<p>My fiancée and I are looking for a good Car...,9.0,101.0,2011-12-28T21:36:43.910,...,<caribbean><cruising><vacations>,4.0,4,2013-02-25T23:52:47.953,CC BY-SA 3.0,NaN,NaN,NaN,NaN,NaN
1,2,1,NaN,2011-06-21T20:22:33.760,43,3255.0,"<p>This was one of our definition questions, b...",13.0,19400.0,2021-04-27T00:47:20.470,...,<guides><extreme-tourism><amazon-river><amazon...,8.0,4,NaN,CC BY-SA 4.0,5.0,NaN,NaN,NaN,NaN
2,3,2,NaN,2011-06-21T20:24:28.080,15,NaN,<p>One way would be to go through an Adventure...,9.0,NaN,NaN,...,NaN,NaN,2,NaN,CC BY-SA 3.0,NaN,2.0,NaN,NaN,NaN
3,4,1,NaN,2011-06-21T20:24:57.160,8,291.0,<p>Singapore Airlines has an all-business clas...,24.0,693.0,2013-01-09T09:55:22.743,...,<loyalty-programs><routes><ewr><singapore-airl...,1.0,1,NaN,CC BY-SA 3.0,NaN,NaN,NaN,NaN,NaN
4,5,1,770.0,2011-06-21T20:25:56.787,14,476.0,<p>Another definition question that interested...,13.0,101.0,2011-12-28T21:36:18.230,...,<romania><transportation>,5.0,0,NaN,CC BY-SA 3.0,2.0,NaN,NaN,NaN,NaN


Now, we can create a function that will select only the needed data. First, we filter out the data and choose only required columns, dropping NaNs in the process. Then, we add a new column, `Year`, storing dates transformed using the `strftime` function. Next, we group and obtain a `Count` column. To continue, we need to select required columns from the `Posts` frame and filter out records not fulfilling the provided condition. Then, we merge two data frames, select needed columns, and drop NaNs and duplicates. In the end, we sort values two times: first with `groupby` and `head(1)`, which is a simple way of selecting the maximum value for each year and removing other values, and secondly to change the order of the data and reset indices.

In [488]:
up_votes_per_year = Votes.loc[Votes["VoteTypeId"] == 2][["PostId", "CreationDate"]].dropna()
up_votes_per_year["Year"] = pd.to_datetime(up_votes_per_year["CreationDate"]).dt.strftime("%Y")
up_votes_per_year["Count"] = up_votes_per_year.groupby(["PostId", "Year"]).transform("count")
df = Posts[["Title", "Id", "PostTypeId"]]
df = df.loc[df["PostTypeId"] == 1]
df = pd.merge(df, up_votes_per_year, left_on="Id", right_on="PostId")
df = df[["Title", "Year", "Count"]].dropna().drop_duplicates()
df = df.sort_values(by="Count", ascending=False).groupby("Year").head(1)
df = df.sort_values(by=["Year"], ascending=True).reset_index(drop=True)
print(df)

                                                Title  Year  Count
0   OK we're all adults here, so really, how on ea...  2011     70
1     How to successfully haggle / bargain in markets  2012     37
2   Why are airline passengers asked to lift up wi...  2013    103
3   How do you know if Americans genuinely/literal...  2014    179
4   Immigration officer that stopped me at the air...  2015    117
5   I don't know my nationality. How can I visit D...  2016    134
6                        Why prohibit engine braking?  2017    177
7   How can I find restaurants in the USA where ti...  2018    119
8   My name causes an issue with any booking! (nam...  2019    263
9   What's the longest distance that can be travel...  2020    110
10  Positive drug test in Singapore: How long do I...  2021     61


In [457]:
print(sql_query.equals(df))

True


## Fifth task
### 5.1 - SQL query

In [460]:
sql_query = pd.read_sql_query("""
SELECT
    Posts.Title,
    VotesByAge2.OldVotes
FROM Posts
JOIN (
    SELECT
        PostId,
        MAX(CASE WHEN VoteDate = 'new' THEN Total ELSE 0 END) NewVotes,
        MAX(CASE WHEN VoteDate = 'old' THEN Total ELSE 0 END) OldVotes,
        SUM(Total) AS Votes
    FROM (
        SELECT
            PostId,
            CASE STRFTIME('%Y', CreationDate)
                WHEN '2021' THEN 'new'
                WHEN '2020' THEN 'new'
                ELSE 'old'
                END VoteDate,
            COUNT(*) AS Total
        FROM Votes
        WHERE VoteTypeId IN (1, 2, 5)
        GROUP BY PostId, VoteDate
    ) AS VotesByAge
    GROUP BY VotesByAge.PostId
    HAVING NewVotes=0
) AS VotesByAge2 ON VotesByAge2.PostId=Posts.ID
WHERE Posts.PostTypeId=1
ORDER BY VotesByAge2.OldVotes DESC
LIMIT 10
""", conn)
print(sql_query)

                                               Title  OldVotes
0                       How to avoid drinking vodka?       200
1         How can I do a "broad" search for flights?       175
2  How to convince airport security that I am not...       119
3        Can I accidentally miss the in-flight food?       114
4  How can black travellers deal with unwanted ha...       108
5             Why are there no seat belts on trains?       106
6  How do you avoid "tourist traps" when travelin...       104
7                         Can I fly with a gold bar?       102
8  Best way to dispose of a carry-on bag in Hong ...        99
9    How to successfully haggle / bargain in markets        97


### 5.2 - Pandas query
First, let's see how the data frames look.

In [461]:
Posts.head()

,Id,PostTypeId,AcceptedAnswerId,CreationDate,Score,ViewCount,Body,OwnerUserId,LastEditorUserId,LastEditDate,...,Tags,AnswerCount,CommentCount,ClosedDate,ContentLicense,FavoriteCount,ParentId,LastEditorDisplayName,CommunityOwnedDate,OwnerDisplayName
0,1,1,393.0,2011-06-21T20:19:34.730,8,578.0,<p>My fiancée and I are looking for a good Car...,9.0,101.0,2011-12-28T21:36:43.910,...,<caribbean><cruising><vacations>,4.0,4,2013-02-25T23:52:47.953,CC BY-SA 3.0,NaN,NaN,NaN,NaN,NaN
1,2,1,NaN,2011-06-21T20:22:33.760,43,3255.0,"<p>This was one of our definition questions, b...",13.0,19400.0,2021-04-27T00:47:20.470,...,<guides><extreme-tourism><amazon-river><amazon...,8.0,4,NaN,CC BY-SA 4.0,5.0,NaN,NaN,NaN,NaN
2,3,2,NaN,2011-06-21T20:24:28.080,15,NaN,<p>One way would be to go through an Adventure...,9.0,NaN,NaN,...,NaN,NaN,2,NaN,CC BY-SA 3.0,NaN,2.0,NaN,NaN,NaN
3,4,1,NaN,2011-06-21T20:24:57.160,8,291.0,<p>Singapore Airlines has an all-business clas...,24.0,693.0,2013-01-09T09:55:22.743,...,<loyalty-programs><routes><ewr><singapore-airl...,1.0,1,NaN,CC BY-SA 3.0,NaN,NaN,NaN,NaN,NaN
4,5,1,770.0,2011-06-21T20:25:56.787,14,476.0,<p>Another definition question that interested...,13.0,101.0,2011-12-28T21:36:18.230,...,<romania><transportation>,5.0,0,NaN,CC BY-SA 3.0,2.0,NaN,NaN,NaN,NaN


In [462]:
Votes.head()

,Id,PostId,VoteTypeId,CreationDate,UserId,BountyAmount
0,1,1,2,2011-06-21T00:00:00.000,NaN,NaN
1,2,1,2,2011-06-21T00:00:00.000,NaN,NaN
2,3,2,2,2011-06-21T00:00:00.000,NaN,NaN
3,4,3,2,2011-06-21T00:00:00.000,NaN,NaN
4,5,5,5,2011-06-21T00:00:00.000,13.0,NaN


Now, we can create a function that will select only the needed data.

In [542]:
VotesByAge = Votes[(Votes["VoteTypeId"] == 1) | Votes["VoteTypeId"] == 2 | (Votes["VoteTypeId"] == 5)][["PostId", "CreationDate"]]
VotesByAge["CreationDate"] = pd.to_datetime(VotesByAge["CreationDate"]).dt.strftime("%Y")
VotesByAge["VoteDate"] = np.select([(VotesByAge["CreationDate"] == "2020") | (VotesByAge["CreationDate"] == "2021"), (VotesByAge["CreationDate"] != "2020") & (VotesByAge["CreationDate"] != "2021")], ["new", "old"])
VotesByAge["Total"] = VotesByAge.groupby(['PostId', 'VoteDate']).transform("count")
new = VotesByAge.loc[VotesByAge["VoteDate"] == "new"].rename(columns={"Total": "NewVotes"})
old = VotesByAge.loc[VotesByAge["VoteDate"] == "old"].rename(columns={"Total": "OldVotes"})
VotesByAge2 = pd.merge(new, old, how="outer", on="PostId").fillna(0)
VotesByAge2 = VotesByAge2[VotesByAge2["VoteDate_x"] == 0].reset_index(drop=True)
VotesByAge2 = VotesByAge2[["PostId", "NewVotes", "OldVotes"]]
df = Posts.loc[Posts["PostTypeId"] == 1][["Title", "Id"]]
df = pd.merge(df, VotesByAge2, left_on="Id", right_on="PostId")
df = df[["Title", "OldVotes"]]
df = df.sort_values(by=["OldVotes"], ascending=False).drop_duplicates().reset_index(drop=True).head(10)
print(df)

                                               Title  OldVotes
0                       How to avoid drinking vodka?     203.0
1         How can I do a "broad" search for flights?     176.0
2  How to convince airport security that I am not...     120.0
3        Can I accidentally miss the in-flight food?     117.0
4                         Can I fly with a gold bar?     114.0
5  How can black travellers deal with unwanted ha...     113.0
6  Best way to dispose of a carry-on bag in Hong ...     110.0
7  How do you avoid "tourist traps" when travelin...     107.0
8             Why are there no seat belts on trains?     107.0
9           How to spot fake reviews on TripAdvisor?      99.0
